In [1]:
import torch
from torch import nn
import torchvision 
from torchvision import datasets,models,transforms
from torchvision.models import alexnet,AlexNet_Weights
from torch.utils.data import dataloader
from torchvision.datasets import ImageFolder 
from torch.utils.data import DataLoader

class CNN(nn.Module):
    def __init__(self, input_shape,hidden_units,output_shape):
        super().__init__()
        self.blk1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2)
        )

        self.blk2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )


        self.clf = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*7*7,out_features=output_shape)
        )

    def forward(self,x):
        x = self.blk1(x)
       # print(f"blk1 shape : {x.shape}")
        x = self.blk2(x)
       # print(f"blk2 shape : {x.shape}")
        x = self.clf(x)
        #print(f"clf shape : {x.shape}")
        return x

train_data = datasets.MNIST(root='data/',download=True,train=True,target_transform=None,transform=transforms.ToTensor())
test_data = datasets.MNIST(root='data/',download=True,train=False,target_transform=None,transform=transforms.ToTensor())
train_loader = DataLoader(dataset=train_data,batch_size=32,shuffle=True)
test_loader = DataLoader(dataset=test_data,batch_size=32,shuffle=False)
# img,label = train_data[0]
# print(img.shape)
# train_features_batch,train_labels_batch = next(iter(train_loader))
# print(train_features_batch.shape)
def train_step(model,train_loader,loss_fn,optimizer,device):
    model.train()
    train_loss,train_acc = 0,0
    for batch,(X,y) in enumerate(train_loader):
        X,y =  X.to(device),y.to(device)
        y_pred = model(X)
        loss = loss_fn(y_pred,y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_acc += (torch.eq(y,y_pred.argmax(dim=1)).sum().item()/ len(y_pred))*100
    train_loss /= len(train_loader)
    train_acc /= len(train_loader)
    print(f"Train loss : {train_loss:.4f} | Train acc : {train_acc:.4f}")

def test_step(model,test_loader,loss_fn,device):
    test_loss,test_acc = 0,0
    model.eval()
    for batch,(X,y) in enumerate(test_loader):
        X,y = X.to(device),y.to(device)
        y_pred = model(X)
        loss = loss_fn(y_pred,y)
        test_acc += (torch.eq(y_pred.argmax(dim=1),y).sum().item() / len(y_pred))*100
        test_loss += loss.item()
    test_loss /= len(test_loader)
    test_acc /= len(test_loader)
    print(f"Test Loss : {test_loss:.4f} | Test acc: {test_acc:.4f}")

epochs = 5
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}")
    train_step(model,train_loader,loss_fn,optimizer,device)
    test_step(model,test_loader,loss_fn,device)num_classes = len(train_data.classes)
model = CNN(input_shape=1,hidden_units=10,output_shape=num_classes).to(device)
# rand_img_tensor = torch.rand(size=(1,28,28))
# print(rand_img_tensor.shape) 
# rand_img_tensor = rand_img_tensor.unsqueeze(dim=0)
# print(rand_img_tensor.shape)
# model(rand_img_tensor.to(device))
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

def train_step(model,train_loader,loss_fn,optimizer,device):
    model.train()
    train_loss,train_acc = 0,0
    for batch,(X,y) in enumerate(train_loader):
        X,y =  X.to(device),y.to(device)
        y_pred = model(X)
        loss = loss_fn(y_pred,y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_acc += (torch.eq(y,y_pred.argmax(dim=1)).sum().item()/ len(y_pred))*100
    train_loss /= len(train_loader)
    train_acc /= len(train_loader)
    print(f"Train loss : {train_loss:.4f} | Train acc : {train_acc:.4f}")

def test_step(model,test_loader,loss_fn,device):
    test_loss,test_acc = 0,0
    model.eval()
    for batch,(X,y) in enumerate(test_loader):
        X,y = X.to(device),y.to(device)
        y_pred = model(X)
        loss = loss_fn(y_pred,y)
        test_acc += (torch.eq(y_pred.argmax(dim=1),y).sum().item() / len(y_pred))*100
        test_loss += loss.item()
    test_loss /= len(test_loader)
    test_acc /= len(test_loader)
    print(f"Test Loss : {test_loss:.4f} | Test acc: {test_acc:.4f}")

epochs = 5
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}")
    train_step(model,train_loader,loss_fn,optimizer,device)
    test_step(model,test_loader,loss_fn,device)

from pathlib import Path 
model_path = Path('model_list')
model_path.mkdir(parents=True,exist_ok=True)
model_name = 'cnn.pth'
model_save_path = model_path / model_name 
torch.save(obj=model.state_dict(),f=model_save_path)

Epoch 1
Train loss : 0.1638 | Train acc : 94.9450
Test Loss : 0.0856 | Test acc: 97.2943
Epoch 2
Train loss : 0.0949 | Train acc : 97.1567
Test Loss : 0.0799 | Test acc: 97.4541
Epoch 3
Train loss : 0.0897 | Train acc : 97.3167
Test Loss : 0.0960 | Test acc: 97.0248
Epoch 4
Train loss : 0.0854 | Train acc : 97.3817
Test Loss : 0.0696 | Test acc: 97.7236
Epoch 5
Train loss : 0.0803 | Train acc : 97.5667
Test Loss : 0.0603 | Test acc: 98.0431


In [3]:
loaded_model = CNN(input_shape=1,hidden_units=10,output_shape=num_classes)
loaded_model.load_state_dict(torch.load(f=model_save_path))
loaded_model = loaded_model.to(device)
train_data = datasets.FashionMNIST(root='data/',download=True,train=True,target_transform=None,transform=transforms.ToTensor())
test_data = datasets.FashionMNIST(root='data/',download=True,train=False,target_transform=None,transform=transforms.ToTensor())
train_loader = DataLoader(dataset=train_data,batch_size=32,shuffle=True)
test_loader = DataLoader(dataset=test_data,batch_size=32)
for X,y in test_loader:
    X,y = X.to(device),y.to(device)
    y_logit = model(X)
    y_pred = torch.softmax(y_logit,dim=1).argmax(dim=1)
    print(f"True label : {y} Predicted label:{y_pred}")

epochs = 5
# for epoch in range(epochs):
#     print(f"Epoch {epoch + 1}")
#     train_step(loaded_model,train_loader,loss_fn,optimizer,device)
#     test_step(loaded_model,test_loader,loss_fn,device) 

def train_step(model,train_loader,loss_fn,optimizer,device):
    model.train()
    train_loss,train_acc = 0,0
    for batch,(X,y) in enumerate(train_loader):
        X,y =  X.to(device),y.to(device)
        y_pred = model(X)
        loss = loss_fn(y_pred,y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_acc += (torch.eq(y,y_pred.argmax(dim=1)).sum().item()/ len(y_pred))*100
    train_loss /= len(train_loader)
    train_acc /= len(train_loader)
    print(f"Train loss : {train_loss:.4f} | Train acc : {train_acc:.4f}")
    return train_loss

def test_step(model,test_loader,loss_fn,device):
    test_loss,test_acc = 0,0
    model.eval()
    for batch,(X,y) in enumerate(test_loader):
        X,y = X.to(device),y.to(device)
        y_pred = model(X)
        loss = loss_fn(y_pred,y)
        test_acc += (torch.eq(y_pred.argmax(dim=1),y).sum().item() / len(y_pred))*100
        test_loss += loss.item()
    test_loss /= len(test_loader)
    test_acc /= len(test_loader)
    print(f"Test Loss : {test_loss:.4f} | Test acc: {test_acc:.4f}")
    return test_loss
import os
checkpoints_dir = 'chkpt'
os.makedirs(checkpoints_dir,exist_ok=True)
best_test_loss = float('inf')
for epoch in range(epochs):
    print(f"Epoch {epoch}---------")
    train_loss = train_step(loaded_model,train_loader,loss_fn,optimizer,device)
    test_loss = test_step(loaded_model,test_loader,loss_fn,device)
    if test_loss < best_test_loss:
        check_pt = {
            'last_loss':test_loss,
            'last_epoch':epochs,
            'model_state_dict':model.state_dict(),
            'optimizer_state_dict':optimizer.state_dict(),
        }
        torch.save(obj=check_pt,f='chkpt/check_pt.pt')
        print("check pt saved")
check_pt = torch.load('chkpt/check_pt.pt') 
loaded_model.load_state_dict(check_pt['model_state_dict'])
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
optimizer.load_state_dict(check_pt['optimizer_state_dict'])
loss = check_pt['last_loss']
epochs = check_pt['last_epoch']


True label : tensor([9, 2, 1, 1, 6, 1, 4, 6, 5, 7, 4, 5, 7, 3, 4, 1, 2, 4, 8, 0, 2, 5, 7, 9,
        1, 4, 6, 0, 9, 3, 8, 8], device='cuda:0') Predicted label:tensor([2, 8, 4, 7, 8, 4, 9, 8, 4, 4, 1, 2, 2, 5, 0, 1, 1, 8, 0, 8, 3, 2, 2, 0,
        4, 8, 0, 8, 2, 5, 2, 0], device='cuda:0')
True label : tensor([3, 3, 8, 0, 7, 5, 7, 9, 6, 1, 3, 7, 6, 7, 2, 1, 2, 2, 4, 4, 5, 8, 2, 2,
        8, 4, 8, 0, 7, 7, 8, 5], device='cuda:0') Predicted label:tensor([5, 8, 8, 8, 4, 2, 4, 2, 8, 7, 5, 2, 0, 2, 3, 0, 9, 0, 0, 1, 2, 0, 5, 0,
        0, 0, 0, 8, 4, 2, 8, 2], device='cuda:0')
True label : tensor([1, 1, 2, 3, 9, 8, 7, 0, 2, 6, 2, 3, 1, 2, 8, 4, 1, 8, 5, 9, 5, 0, 3, 2,
        0, 6, 5, 3, 6, 7, 1, 8], device='cuda:0') Predicted label:tensor([4, 1, 5, 7, 2, 5, 1, 3, 0, 5, 8, 1, 7, 0, 8, 3, 7, 8, 8, 5, 2, 3, 5, 8,
        8, 0, 2, 6, 0, 4, 4, 0], device='cuda:0')
True label : tensor([0, 1, 4, 2, 3, 6, 7, 2, 7, 8, 5, 9, 9, 4, 2, 5, 7, 0, 5, 2, 8, 6, 7, 8,
        0, 0, 9, 9, 3, 0, 8, 4], device=

In [25]:
from torchvision.models import alexnet,AlexNet_Weights
weights = AlexNet_Weights.DEFAULT
weights
auto_transform = weights.transforms()
auto_transform
device = "cuda" if torch.cuda.is_available() else "cpu"

train_data = ImageFolder(root=r'C:\Users\rsurs\OneDrive\Documents\University_Study_Material\3rd year\6th sem\DL_lab\Lab-6\cats_and_dogs_filtered\train',
                         transform=auto_transform)
test_data = ImageFolder(root=r'C:\Users\rsurs\OneDrive\Documents\University_Study_Material\3rd year\6th sem\DL_lab\Lab-6\cats_and_dogs_filtered\validation',
                         transform=auto_transform)
train_loader = DataLoader(dataset=train_data,batch_size=32,shuffle=True)
test_loader = DataLoader(dataset=test_data,batch_size=32)
model = alexnet(weights=weights).to(device)
num_classes = len(train_data.classes)
print(num_classes)
model
img,label = train_data[0]
print(img.shape)

for param in model.features.parameters():
    param.requires_grad = False

from torchinfo import summary

summary(model,input_size=(32,3,224,224))

num_features = model.classifier[6].in_features
model.classifier[6] = nn.Sequential(nn.Dropout(p=0.2),
                                    nn.Linear(in_features=num_features,out_features=num_classes)).to(device)
model
def train_step(model,train_loader,loss_fn,optimizer,device):
    model.train()
    train_loss,train_acc = 0,0
    for batch,(X,y) in enumerate(train_loader):
        X,y =  X.to(device),y.to(device)
        y_pred = model(X)
        loss = loss_fn(y_pred,y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_acc += (torch.eq(y,y_pred.argmax(dim=1)).sum().item()/ len(y_pred))*100
    train_loss /= len(train_loader)
    train_acc /= len(train_loader)
    print(f"Train loss : {train_loss:.4f} | Train acc : {train_acc:.4f}")

def test_step(model,test_loader,loss_fn,device):
    test_loss,test_acc = 0,0
    model.eval()
    for batch,(X,y) in enumerate(test_loader):
        X,y = X.to(device),y.to(device)
        y_pred = model(X)
        loss = loss_fn(y_pred,y)
        test_acc += (torch.eq(y_pred.argmax(dim=1),y).sum().item() / len(y_pred))*100
        test_loss += loss.item()
    test_loss /= len(test_loader)
    test_acc /= len(test_loader)
    print(f"Test Loss : {test_loss:.4f} | Test acc: {test_acc:.4f}")

epochs = 5
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}")
    train_step(model,train_loader,loss_fn,optimizer,device)
    test_step(model,test_loader,loss_fn,device)


2
torch.Size([3, 224, 224])
Epoch 1
Train loss : 0.7652 | Train acc : 54.4643
Test Loss : 0.7176 | Test acc: 57.0312
Epoch 2
Train loss : 0.7561 | Train acc : 55.7540
Test Loss : 0.7176 | Test acc: 57.0312
Epoch 3
Train loss : 0.7662 | Train acc : 55.0099
Test Loss : 0.7176 | Test acc: 57.0312
Epoch 4
Train loss : 0.7616 | Train acc : 55.3075
Test Loss : 0.7176 | Test acc: 57.0312
Epoch 5
Train loss : 0.7611 | Train acc : 55.8036
Test Loss : 0.7176 | Test acc: 57.0312


L2 and L1 reg

In [14]:
import torch
from torch import nn
import torchvision 
from torchvision import datasets,models,transforms
from torchvision.models import alexnet,AlexNet_Weights
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder 
from torch.utils.data import DataLoader
 
class CatsAndDogs(nn.Module):
    def __init__(self, input_shape,hidden_units,output_shape):
        super().__init__()
        self.blk1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2)
        )

        self.blk2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )


        self.clf = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*25*25,out_features=output_shape)
        )
    
    def forward(self,x):
        x = self.blk1(x)
        #print(f"blk 1 shape: {x.shape}")
        x = self.blk2(x)
        #print(f"blk 2 shape: {x.shape}")
        x = self.clf(x)
        #print(f"clf shape: {x.shape}")
        return x

transform = transforms.Compose([
    transforms.Resize(100),  # Resize shortest side to 100 pixels
    transforms.CenterCrop(100),  # Crop a square in the center
    transforms.ToTensor()
])

train_data = ImageFolder(root=r'C:\Users\rsurs\OneDrive\Documents\University_Study_Material\3rd year\6th sem\DL_lab\Lab-6\cats_and_dogs_filtered\train',
                        transform=transform)
test_data = ImageFolder(root=r'C:\Users\rsurs\OneDrive\Documents\University_Study_Material\3rd year\6th sem\DL_lab\Lab-6\cats_and_dogs_filtered\validation',
                        transform=transform)
train_loader = DataLoader(dataset=train_data,batch_size=32,shuffle=True)
test_loader = DataLoader(dataset=test_data,batch_size=32)
img,label = train_data[0]
print(img.shape)
num_classes = len(train_data.classes)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CatsAndDogs(input_shape=3,hidden_units=10,output_shape=num_classes).to(device)
rand_img_tensor = torch.rand(size=(3,100,100))
rand_img_tensor = rand_img_tensor.unsqueeze(dim=0)
model(rand_img_tensor.to(device))
loss_fn = nn.CrossEntropyLoss()
optimizer= torch.optim.Adam(model.parameters(),lr=0.01,weight_decay=0.001)

def train_step(model,train_loader,loss_fn,optimizer,device):
    train_acc,train_loss = 0,0
    model.train()
    for batch,(X,y) in enumerate(train_loader):
        X,y = X.to(device),y.to(device)
        y_pred = model(X)
        loss = loss_fn(y_pred,y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_acc += (torch.eq(y_pred.argmax(dim=1),y).sum().item()/len(y_pred))*100
        y_pred_class = torch.softmax(y_pred,dim=1).argmax(dim=1)
    train_loss /= len(train_loader)
    train_acc /= len(train_loader)
    print(f"Train loss: {train_loss:.4f} | Train acc: {train_acc:.4f}") 


def test_step(model,test_loader,loss_fn,device):
    test_acc,test_loss = 0,0
    model.eval()
    with torch.inference_mode():
        for batch,(X,y) in enumerate(test_loader):
            X,y = X.to(device),y.to(device)
            y_pred = model(X)
            loss = loss_fn(y_pred,y)
            test_loss += loss.item()
            test_acc += (torch.eq(y_pred.argmax(dim=1),y).sum().item()/len(y_pred))*100
            y_pred_class = torch.softmax(y_pred,dim=1).argmax(dim=1)
        test_loss /= len(train_loader)
        test_acc /= len(train_loader)
        print(f"Test loss: {test_loss:.4f} | Train acc: {test_acc:.4f}") 

epochs = 5
for epoch in range(epochs):
    print(f"Epoch {epoch}:------")
    train_step(model,train_loader,loss_fn,optimizer,device)
    test_step(model,test_loader,loss_fn,device)

torch.Size([3, 100, 100])
Epoch 0:------
Train loss: 0.7011 | Train acc: 48.9583
Test loss: 0.3520 | Train acc: 25.9921
Epoch 1:------
Train loss: 0.6937 | Train acc: 48.8591
Test loss: 0.3527 | Train acc: 24.8016
Epoch 2:------
Train loss: 0.6943 | Train acc: 49.2560
Test loss: 0.3520 | Train acc: 25.9921
Epoch 3:------
Train loss: 0.6937 | Train acc: 50.1984
Test loss: 0.3522 | Train acc: 24.8016
Epoch 4:------
Train loss: 0.6937 | Train acc: 49.2063
Test loss: 0.3525 | Train acc: 24.8016


In [15]:
import torch
from torch import nn
import torchvision 
from torchvision import datasets,models,transforms
from torchvision.models import alexnet,AlexNet_Weights
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder 
from torch.utils.data import DataLoader
 
class CatsAndDogs(nn.Module):
    def __init__(self, input_shape,hidden_units,output_shape):
        super().__init__()
        self.blk1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2)
        )

        self.blk2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )


        self.clf = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*25*25,out_features=output_shape)
        )
    
    def forward(self,x):
        x = self.blk1(x)
        #print(f"blk 1 shape: {x.shape}")
        x = self.blk2(x)
        #print(f"blk 2 shape: {x.shape}")
        x = self.clf(x)
        #print(f"clf shape: {x.shape}")
        return x

transform = transforms.Compose([
    transforms.Resize(100),  # Resize shortest side to 100 pixels
    transforms.CenterCrop(100),  # Crop a square in the center
    transforms.ToTensor()
])

train_data = ImageFolder(root=r'C:\Users\rsurs\OneDrive\Documents\University_Study_Material\3rd year\6th sem\DL_lab\Lab-6\cats_and_dogs_filtered\train',
                        transform=transform)
test_data = ImageFolder(root=r'C:\Users\rsurs\OneDrive\Documents\University_Study_Material\3rd year\6th sem\DL_lab\Lab-6\cats_and_dogs_filtered\validation',
                        transform=transform)
train_loader = DataLoader(dataset=train_data,batch_size=32,shuffle=True)
test_loader = DataLoader(dataset=test_data,batch_size=32)
img,label = train_data[0]
print(img.shape)
num_classes = len(train_data.classes)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CatsAndDogs(input_shape=3,hidden_units=10,output_shape=num_classes).to(device)
rand_img_tensor = torch.rand(size=(3,100,100))
rand_img_tensor = rand_img_tensor.unsqueeze(dim=0)
model(rand_img_tensor.to(device))
loss_fn = nn.CrossEntropyLoss()
optimizer= torch.optim.Adam(model.parameters(),lr=0.01)
lambda_ = 0.01
def train_step(model,train_loader,loss_fn,optimizer,device):
    train_acc,train_loss = 0,0
    model.train()
    for batch,(X,y) in enumerate(train_loader):
        X,y = X.to(device),y.to(device)
        y_pred = model(X)
        loss = loss_fn(y_pred,y)
        l2_reg = 0.0
        for param in model.parameters():
            l2_reg = torch.norm(param,p=2)
        loss += 0.5*lambda_*l2_reg
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_acc += (torch.eq(y_pred.argmax(dim=1),y).sum().item()/len(y_pred))*100
        y_pred_class = torch.softmax(y_pred,dim=1).argmax(dim=1)
    train_loss /= len(train_loader)
    train_acc /= len(train_loader)
    print(f"Train loss: {train_loss:.4f} | Train acc: {train_acc:.4f}") 


def test_step(model,test_loader,loss_fn,device):
    test_acc,test_loss = 0,0
    model.eval()
    with torch.inference_mode():
        for batch,(X,y) in enumerate(test_loader):
            X,y = X.to(device),y.to(device)
            y_pred = model(X)
            loss = loss_fn(y_pred,y)
            test_loss += loss.item()
            test_acc += (torch.eq(y_pred.argmax(dim=1),y).sum().item()/len(y_pred))*100
            y_pred_class = torch.softmax(y_pred,dim=1).argmax(dim=1)
        test_loss /= len(train_loader)
        test_acc /= len(train_loader)
        print(f"Test loss: {test_loss:.4f} | Train acc: {test_acc:.4f}") 

epochs = 5
for epoch in range(epochs):
    print(f"Epoch {epoch}:------")
    train_step(model,train_loader,loss_fn,optimizer,device)
    test_step(model,test_loader,loss_fn,device)

torch.Size([3, 100, 100])
Epoch 0:------
Train loss: 0.7084 | Train acc: 49.8016
Test loss: 0.3522 | Train acc: 24.8016
Epoch 1:------
Train loss: 0.6935 | Train acc: 48.8095
Test loss: 0.3521 | Train acc: 24.8016
Epoch 2:------
Train loss: 0.6934 | Train acc: 48.8591
Test loss: 0.3522 | Train acc: 24.8016
Epoch 3:------
Train loss: 0.6934 | Train acc: 51.0417
Test loss: 0.3519 | Train acc: 25.9921
Epoch 4:------
Train loss: 0.6936 | Train acc: 49.2560
Test loss: 0.3522 | Train acc: 24.8016


In [16]:
import torch
from torch import nn
import torchvision 
from torchvision import datasets,models,transforms
from torchvision.models import alexnet,AlexNet_Weights
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder 
from torch.utils.data import DataLoader
 
class CatsAndDogs(nn.Module):
    def __init__(self, input_shape,hidden_units,output_shape):
        super().__init__()
        self.blk1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2)
        )

        self.blk2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )


        self.clf = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*25*25,out_features=output_shape)
        )
    
    def forward(self,x):
        x = self.blk1(x)
        #print(f"blk 1 shape: {x.shape}")
        x = self.blk2(x)
        #print(f"blk 2 shape: {x.shape}")
        x = self.clf(x)
        #print(f"clf shape: {x.shape}")
        return x

transform = transforms.Compose([
    transforms.Resize(100),  # Resize shortest side to 100 pixels
    transforms.CenterCrop(100),  # Crop a square in the center
    transforms.ToTensor()
])

train_data = ImageFolder(root=r'C:\Users\rsurs\OneDrive\Documents\University_Study_Material\3rd year\6th sem\DL_lab\Lab-6\cats_and_dogs_filtered\train',
                        transform=transform)
test_data = ImageFolder(root=r'C:\Users\rsurs\OneDrive\Documents\University_Study_Material\3rd year\6th sem\DL_lab\Lab-6\cats_and_dogs_filtered\validation',
                        transform=transform)
train_loader = DataLoader(dataset=train_data,batch_size=32,shuffle=True)
test_loader = DataLoader(dataset=test_data,batch_size=32)
img,label = train_data[0]
print(img.shape)
num_classes = len(train_data.classes)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CatsAndDogs(input_shape=3,hidden_units=10,output_shape=num_classes).to(device)
rand_img_tensor = torch.rand(size=(3,100,100))
rand_img_tensor = rand_img_tensor.unsqueeze(dim=0)
model(rand_img_tensor.to(device))
loss_fn = nn.CrossEntropyLoss()
optimizer= torch.optim.Adam(model.parameters(),lr=0.01)
lambda_ = 0.01
def train_step(model,train_loader,loss_fn,optimizer,device):
    train_acc,train_loss = 0,0
    model.train()
    for batch,(X,y) in enumerate(train_loader):
        X,y = X.to(device),y.to(device)
        y_pred = model(X)
        loss = loss_fn(y_pred,y)
        l1_reg = 0.0
        for param in model.parameters():
            l1_reg = torch.norm(param,p=1)
        loss += lambda_*0.5 * l1_reg
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_acc += (torch.eq(y_pred.argmax(dim=1),y).sum().item()/len(y_pred))*100
        y_pred_class = torch.softmax(y_pred,dim=1).argmax(dim=1)
    train_loss /= len(train_loader)
    train_acc /= len(train_loader)
    print(f"Train loss: {train_loss:.4f} | Train acc: {train_acc:.4f}") 


def test_step(model,test_loader,loss_fn,device):
    test_acc,test_loss = 0,0
    model.eval()
    with torch.inference_mode():
        for batch,(X,y) in enumerate(test_loader):
            X,y = X.to(device),y.to(device)
            y_pred = model(X)
            loss = loss_fn(y_pred,y)
            test_loss += loss.item()
            test_acc += (torch.eq(y_pred.argmax(dim=1),y).sum().item()/len(y_pred))*100
            y_pred_class = torch.softmax(y_pred,dim=1).argmax(dim=1)
        test_loss /= len(train_loader)
        test_acc /= len(train_loader)
        print(f"Test loss: {test_loss:.4f} | Train acc: {test_acc:.4f}") 

epochs = 5
for epoch in range(epochs):
    print(f"Epoch {epoch}:------")
    train_step(model,train_loader,loss_fn,optimizer,device)
    test_step(model,test_loader,loss_fn,device)

torch.Size([3, 100, 100])
Epoch 0:------
Train loss: 0.7641 | Train acc: 49.0575
Test loss: 0.3519 | Train acc: 25.9921
Epoch 1:------
Train loss: 0.6933 | Train acc: 50.5456
Test loss: 0.3522 | Train acc: 24.8016
Epoch 2:------
Train loss: 0.6936 | Train acc: 50.3472
Test loss: 0.3521 | Train acc: 25.9921
Epoch 3:------
Train loss: 0.6933 | Train acc: 50.1984
Test loss: 0.3520 | Train acc: 25.9921
Epoch 4:------
Train loss: 0.6940 | Train acc: 48.9583
Test loss: 0.3519 | Train acc: 25.9921


DropOut

In [18]:
import torch
from torch import nn
import torchvision 
from torchvision import datasets,models,transforms
from torchvision.models import alexnet,AlexNet_Weights
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder 
from torch.utils.data import DataLoader
 
class CatsAndDogs(nn.Module):
    def __init__(self, input_shape,hidden_units,output_shape):
        super().__init__()
        self.blk1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2),
            nn.Dropout(p=0.5)
        )

        self.blk2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(p=0.5)
        )


        self.clf = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=hidden_units*25*25,out_features=output_shape)
        )
    
    def forward(self,x):
        x = self.blk1(x)
        #print(f"blk 1 shape: {x.shape}")
        x = self.blk2(x)
        #print(f"blk 2 shape: {x.shape}")
        x = self.clf(x)
        #print(f"clf shape: {x.shape}")
        return x

transform = transforms.Compose([
    transforms.Resize(100),  # Resize shortest side to 100 pixels
    transforms.CenterCrop(100),  # Crop a square in the center
    transforms.ToTensor()
])

train_data = ImageFolder(root=r'C:\Users\rsurs\OneDrive\Documents\University_Study_Material\3rd year\6th sem\DL_lab\Lab-6\cats_and_dogs_filtered\train',
                        transform=transform)
test_data = ImageFolder(root=r'C:\Users\rsurs\OneDrive\Documents\University_Study_Material\3rd year\6th sem\DL_lab\Lab-6\cats_and_dogs_filtered\validation',
                        transform=transform)
train_loader = DataLoader(dataset=train_data,batch_size=32,shuffle=True)
test_loader = DataLoader(dataset=test_data,batch_size=32)
img,label = train_data[0]
print(img.shape)
num_classes = len(train_data.classes)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CatsAndDogs(input_shape=3,hidden_units=10,output_shape=num_classes).to(device)
rand_img_tensor = torch.rand(size=(3,100,100))
rand_img_tensor = rand_img_tensor.unsqueeze(dim=0)
model(rand_img_tensor.to(device))
loss_fn = nn.CrossEntropyLoss()
optimizer= torch.optim.Adam(model.parameters(),lr=0.01)

def train_step(model,train_loader,loss_fn,optimizer,device):
    train_acc,train_loss = 0,0
    model.train()
    for batch,(X,y) in enumerate(train_loader):
        X,y = X.to(device),y.to(device)
        y_pred = model(X)
        loss = loss_fn(y_pred,y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_acc += (torch.eq(y_pred.argmax(dim=1),y).sum().item()/len(y_pred))*100
        y_pred_class = torch.softmax(y_pred,dim=1).argmax(dim=1)
    train_loss /= len(train_loader)
    train_acc /= len(train_loader)
    print(f"Train loss: {train_loss:.4f} | Train acc: {train_acc:.4f}") 


def test_step(model,test_loader,loss_fn,device):
    test_acc,test_loss = 0,0
    model.eval()
    with torch.inference_mode():
        for batch,(X,y) in enumerate(test_loader):
            X,y = X.to(device),y.to(device)
            y_pred = model(X)
            loss = loss_fn(y_pred,y)
            test_loss += loss.item()
            test_acc += (torch.eq(y_pred.argmax(dim=1),y).sum().item()/len(y_pred))*100
            y_pred_class = torch.softmax(y_pred,dim=1).argmax(dim=1)
        test_loss /= len(train_loader)
        test_acc /= len(train_loader)
        print(f"Test loss: {test_loss:.4f} | Train acc: {test_acc:.4f}") 

epochs = 5
for epoch in range(epochs):
    print(f"Epoch {epoch}:------")
    train_step(model,train_loader,loss_fn,optimizer,device)
    test_step(model,test_loader,loss_fn,device)

torch.Size([3, 100, 100])
Epoch 0:------
Train loss: 0.7004 | Train acc: 50.1488
Test loss: 0.3522 | Train acc: 24.8016
Epoch 1:------
Train loss: 0.6939 | Train acc: 49.2063
Test loss: 0.3530 | Train acc: 24.8016
Epoch 2:------
Train loss: 0.6941 | Train acc: 48.3631
Test loss: 0.3520 | Train acc: 25.9921
Epoch 3:------
Train loss: 0.6936 | Train acc: 47.7679
Test loss: 0.3520 | Train acc: 25.9921
Epoch 4:------
Train loss: 0.6943 | Train acc: 49.7520
Test loss: 0.3519 | Train acc: 25.9921


Early stopping 

In [19]:
import torch
from torch import nn
import torchvision 
from torchvision import datasets,models,transforms
from torchvision.models import alexnet,AlexNet_Weights
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder 
from torch.utils.data import DataLoader
 
class CatsAndDogs(nn.Module):
    def __init__(self, input_shape,hidden_units,output_shape):
        super().__init__()
        self.blk1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2),
            nn.Dropout(p=0.5)
        )

        self.blk2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(p=0.5)
        )


        self.clf = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=hidden_units*25*25,out_features=output_shape)
        )
    
    def forward(self,x):
        x = self.blk1(x)
        #print(f"blk 1 shape: {x.shape}")
        x = self.blk2(x)
        #print(f"blk 2 shape: {x.shape}")
        x = self.clf(x)
        #print(f"clf shape: {x.shape}")
        return x

transform = transforms.Compose([
    transforms.Resize(100),  # Resize shortest side to 100 pixels
    transforms.CenterCrop(100),  # Crop a square in the center
    transforms.ToTensor()
])

train_data = ImageFolder(root=r'C:\Users\rsurs\OneDrive\Documents\University_Study_Material\3rd year\6th sem\DL_lab\Lab-6\cats_and_dogs_filtered\train',
                        transform=transform)
test_data = ImageFolder(root=r'C:\Users\rsurs\OneDrive\Documents\University_Study_Material\3rd year\6th sem\DL_lab\Lab-6\cats_and_dogs_filtered\validation',
                        transform=transform)
train_loader = DataLoader(dataset=train_data,batch_size=32,shuffle=True)
test_loader = DataLoader(dataset=test_data,batch_size=32)
img,label = train_data[0]
print(img.shape)
num_classes = len(train_data.classes)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CatsAndDogs(input_shape=3,hidden_units=10,output_shape=num_classes).to(device)
rand_img_tensor = torch.rand(size=(3,100,100))
rand_img_tensor = rand_img_tensor.unsqueeze(dim=0)
model(rand_img_tensor.to(device))
loss_fn = nn.CrossEntropyLoss()
optimizer= torch.optim.Adam(model.parameters(),lr=0.01)
best_test_loss = float('inf')
curr_pt = 0
pt = 5
for epoch in range(epochs):
    train_acc,train_loss = 0,0
    model.train()
    for batch,(X,y) in enumerate(train_loader):
        X,y = X.to(device),y.to(device)
        y_pred = model(X)
        loss = loss_fn(y_pred,y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_acc += (torch.eq(y_pred.argmax(dim=1),y).sum().item()/len(y_pred))*100
        y_pred_class = torch.softmax(y_pred,dim=1).argmax(dim=1)
    train_loss /= len(train_loader)
    train_acc /= len(train_loader)
    test_acc,test_loss = 0,0
    model.eval()
    with torch.inference_mode():
        for batch,(X,y) in enumerate(test_loader):
            X,y = X.to(device),y.to(device)
            y_pred = model(X)
            loss = loss_fn(y_pred,y)
            test_loss += loss.item()
            test_acc += (torch.eq(y_pred.argmax(dim=1),y).sum().item()/len(y_pred))*100
            y_pred_class = torch.softmax(y_pred,dim=1).argmax(dim=1)
        test_loss /= len(train_loader)
        test_acc /= len(train_loader)
        if test_loss < best_test_loss:
            best_test_loss = test_loss
            curr_pt = 0
            torch.save(obj=model.state_dict(),f='best_model.pth')
        else:
            curr_pt += 1
        if curr_pt > pt:
            print("Early stopping! No improvement for {} epochs.".format(pt))
            break

    print(f"Epoch {epoch+1}:")
    print(f"Train Loss: {train_loss:.4f} | Train Accuracy: {train_acc:.4f}")
    print(f"Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")

torch.Size([3, 100, 100])
Epoch 1:
Train Loss: 0.7113 | Train Accuracy: 49.3552
Test Loss: 0.3522 | Test Accuracy: 24.8016
Epoch 2:
Train Loss: 0.6935 | Train Accuracy: 46.8254
Test Loss: 0.3522 | Test Accuracy: 24.8016
Epoch 3:
Train Loss: 0.6939 | Train Accuracy: 48.8095
Test Loss: 0.3523 | Test Accuracy: 24.8016
Epoch 4:
Train Loss: 0.6935 | Train Accuracy: 49.8512
Test Loss: 0.3519 | Test Accuracy: 25.9921
Epoch 5:
Train Loss: 0.6937 | Train Accuracy: 50.1984
Test Loss: 0.3526 | Test Accuracy: 24.8016
